In [48]:
import cv2
import numpy as np
import math
import time
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression

In [103]:
def paintLine(frame, x1,y1,x2,y2):
    lined_frame = cv2.line(frame,(x1,y1), (x2,y2), (255,0,0), 10)
    line_ang = CalculeAng(x1,y1,x2,y2)
    frame_with_ang = WriteAng(lined_frame, line_ang)
    return frame_with_ang

In [112]:
def CalculeAng(x1,y1,x2,y2):
    ang = math.atan2(x2-x1,y2-y1)
    pi = math.pi
    deg_ang = (ang*180)/pi-90
    return deg_ang

In [105]:
def WriteAng(frame, ang):
    font = cv2.FONT_HERSHEY_SIMPLEX
    ang_frame = cv2.putText(frame, str(ang), org=(100,100), fontFace=font, fontScale = 2, color=(255,255,255), thickness=3, lineType=cv2.LINE_AA)
    return ang_frame

In [13]:
def edgeDet(frame):
    med_val = np.median(frame)
    upper = int(min(255,1.3*med_val))
    lower = int(max(0,0.7*med_val))
    edged_frame = cv2.Canny(image=blurred_frame,threshold1=lower-50, threshold2=upper-150)
    return edged_frame

In [14]:
def ResizeWithAspectRatio(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    return cv2.resize(image, dim, interpolation=inter)

In [228]:
def find_centroids(dst):
    ret, dst = cv2.threshold(dst,125, 255, cv2.THRESH_BINARY)
    dst = np.uint8(dst)

    # find centroids
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)
    # define the criteria to stop and refine the corners
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 
                0.001)
    corners = cv2.cornerSubPix(gray,np.float32(centroids),(5,5), 
              (-1,-1),criteria)
    return corners

In [241]:
cap = cv2.VideoCapture('PilotView.mp4')
if cap.isOpened() == False:
    print('Error file not found')
while cap.isOpened():
    ret,frame = cap.read()

    if ret == True:
        #WRITER 20 FPS
        time.sleep(1/100)
        blurred_frame = cv2.blur(frame,ksize=(7,7))
        gray = cv2.cvtColor(blurred_frame, cv2.COLOR_RGB2GRAY)
        thresh, bw_frame = cv2.threshold(gray, 175, 255, cv2.THRESH_BINARY)
        
        gray = np.float32(bw_frame)
        dst = cv2.cornerHarris(gray,20,1,0.1)
        dst = cv2.dilate(dst,None)
        corners = find_centroids(dst)
        splitted = np.hsplit(corners,2)
        x = splitted[0]
        y = splitted[1]
        medY = np.mean(y)+150
        #fixed_corners = corners.copy()
        fixed_corners = np.empty((0,2), dtype=int)
        
        i = 0
        #delete bad corners
        for corner in corners:
            #dst[int(corner[1]), int(corner[0])] = [255,0,0]
            if ((corner[1] > medY + 150) or (corner[1] < medY - 150)):
                #fixed_corners = np.delete(fixed_corners, i,0)
                i+=1
                continue
            temp1 = corner[0]
            temp2 = corner[1]
            fixed_corners = np.append(fixed_corners,np.array([[temp1,temp2]]), 0)
            #dst = cv2.circle(frame, (corner[0],corner[1]),10,(0,0,255),thickness=-1)
            i += 1
            
            
        for corner2 in fixed_corners:
            tempx=corner2[0]
            tempy=corner2[1]
            dst = cv2.circle(frame, (int(tempx),int(tempy)),10,(0,0,255),thickness=-1)
        splitted = np.hsplit(fixed_corners,2)
        x = splitted[0]
        y = splitted[1]
        
        model = LinearRegression().fit(x,y)
        m = model.coef_
        b = model.intercept_
        x1 = 0
        y1 = m[0][0]*x1 + b[0]
        x2 = dst.shape[1]
        y2 = m[0][0]*x2 + b[0]
        #final_frame = cv2.line(dst, (x1,y1), (x2, y2), (0, 0, 255),thickness=20)
        final_frame = paintLine(dst,x1,int(y1),x2,int(y2))
        #thresh, bw_frame = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
        #edged_frame = edgeDet(bw_frame)
        #data_frame = paintLine(edged_frame)
        fixed_size_frame = ResizeWithAspectRatio(final_frame, width=800)
        cv2.imshow('frameVid', fixed_size_frame)

        if cv2.waitKey(10) & 0xFF ==ord('q'):
            break

    else:
        break
        
cap.release()
cv2.destroyAllWindows() 
